In [1]:
import json
import requests

# Using SciPy for distance.



In [2]:
import math 

import scipy.spatial.distance

def ruin_distance(ruin1, ruin2):
    return scipy.spatial.distance.euclidean(ruin1['systemCoordinates'], ruin2['systemCoordinates'])



# Canonn Technology API Documentation is here

https://api.canonn.technology/api/docs/#/

This section fetches all of the data and provides helper functions

In [3]:
api_url_base = 'https://api.canonn.technology/api/v1/'

#headers = {'Content-Type': 'application/json'}


In [4]:
api_url_base = 'https://api.canonn.technology/api/v1/'

def get_from_site(url_ext):
    """Grabs data from the API / url_ext
    Returns None if 200. not much other error handling 
    """
    api_url = '{0}{1}'.format(api_url_base,url_ext)

    response = requests.get(api_url)

    if response.status_code == 200:
        return json.loads(response.content.decode('utf-8'))
    else:
        return None

In [5]:
def get_obelisks():
    return get_from_site('maps/scandata')

def get_ruins_sites():
    return get_from_site('maps/systemoverview')

def get_categories():
    return get_from_site('codex/categories')

def get_codex_data():
    return get_from_site('codex/data')

def get_ruin_types():
    return get_from_site('ruins/types')

def get_all_ruins():
    return get_from_site('ruinsites')

In [6]:
def get_ruin_info(id):
    return get_from_site('maps/ruininfo/%d' % id) 

def get_active_obelisks(id):
    return get_from_site('ruinsites/%d/activeobelisks' % id)

In [7]:
ruin_types = get_ruin_types()
ruins = get_ruins_sites()
obelisks = get_obelisks()
codex_data = get_codex_data()

# all_codex_data is the set of all codex data that we need to find 
# in a set so we can subtract from it along the way to find what's left
all_codex_data = set([x['id'] for x in codex_data])

all_ruins = get_all_ruins()

In [8]:
# Need to filter out all ruin id's that are >10000 - or just drop the last 3 that are special
# Could also filter out edsmBodyLink = None 
# Risky -- says that no ruin will be exactly at longitude 0.0 .. probably good enough 

all_ruins_trimmed = [x for x in all_ruins if (x['longitude'] != 0.0)]


In [16]:
# Long running command - probably should rate limit the get API.
all_ruin_info = [get_ruin_info(ruin['id']) for ruin in all_ruins_trimmed]

In [19]:
# Long running command
all_obelisk_info = [get_active_obelisks(ruin['id']) for ruin in all_ruins_trimmed]

In [63]:
def get_obelisk_group_codex_data(id):
    """Relies upon all_obelisk_info being populated
    returns the sorted set of codex data that this particular obelisk group handles
    """
    obelisk_group = all_obelisk_info[id]
    if obelisk_group != None and len(obelisk_group) > 0:
        return sorted(set([x['codexdataId'] for x in obelisk_group if (x['isBroken'] == False and x['codexdataId'] != None)]))
    else:
        return set([])

In [65]:
# Off by 1...
for i in all_ruins_trimmed:
    print(i['id'])
    print(get_obelisk_group_codex_data(i['id']))
#    print(get_obelisk_group_codex_data(i['id']))

1
[8, 27, 28, 54, 55, 88, 90, 91]
2
[5, 6, 20, 21, 24, 25, 40, 42, 43, 44]
3
[29, 32, 52, 54, 68, 90, 93]
4
[1, 2, 4, 22, 23, 24, 25, 42, 44, 64, 67, 84, 85]
5
[26, 48, 51, 71, 72, 88, 90, 91]
6
[2, 6, 20, 21, 23, 40, 41, 44, 63, 64, 65, 66, 84, 85, 86, 87]
7
[26, 27, 31, 47, 54, 88, 90, 91]
8
[13, 14, 15, 17, 18, 33, 34, 35, 36, 38, 56, 57, 59, 60, 78, 95, 98, 99, 100]
9
[8, 27, 49, 50, 73, 74, 88, 90, 94]
10
[23, 24, 25, 40, 42, 43, 44, 45, 62, 63, 64, 65, 82, 83, 84, 85, 86]
11
[2, 3, 6, 20, 22, 23, 24, 25, 40, 42, 45, 62, 63, 66, 82, 83, 84, 87]
12
[3, 4, 6, 21, 22, 25, 45, 46, 63, 64, 65, 84, 86]
13
[14, 16, 17, 18, 19, 35, 36, 37, 38, 56, 57, 59, 60, 95, 96, 97, 99, 100, 101]
14
[13, 14, 19, 34, 37, 38, 39, 56, 59, 75, 76, 78, 79, 81, 97, 98, 99]
15
[15, 17, 35, 36, 37, 56, 59, 95, 96, 97, 99, 100, 101]
16
[28, 31, 48, 49, 53, 89, 93, 94]
17
[29, 48, 49, 53, 72, 89, 92, 94]
18
[29, 47, 51, 52, 53, 55, 73, 88, 90]
19
[15, 16, 38, 39, 56, 58, 80, 98]
20
[8, 11, 12, 26, 49, 53, 88, 

IndexError: list index out of range

In [67]:
print(len(all_obelisk_info))

226


In [24]:
print(all_obelisk_info[2])

[{'id': 38, 'obeliskgroupId': 2, 'number': 18, 'isBroken': False, 'codexdataId': 20, 'isVerified': True, 'created': '2017-03-19T20:59:30', 'updated': '2017-07-11T18:21:04'}, {'id': 40, 'obeliskgroupId': 2, 'number': 20, 'isBroken': False, 'codexdataId': 21, 'isVerified': True, 'created': '2017-03-19T20:59:30', 'updated': '2017-07-11T18:21:04'}, {'id': 48, 'obeliskgroupId': 3, 'number': 7, 'isBroken': False, 'codexdataId': 24, 'isVerified': True, 'created': '2017-03-19T20:59:30', 'updated': '2017-07-12T14:08:38'}, {'id': 53, 'obeliskgroupId': 3, 'number': 12, 'isBroken': False, 'codexdataId': 25, 'isVerified': False, 'created': '2017-03-19T20:59:30', 'updated': '2017-03-19T20:59:30'}, {'id': 62, 'obeliskgroupId': 4, 'number': 7, 'isBroken': False, 'codexdataId': 24, 'isVerified': False, 'created': '2017-03-19T20:59:30', 'updated': '2017-07-11T18:21:04'}, {'id': 67, 'obeliskgroupId': 4, 'number': 12, 'isBroken': False, 'codexdataId': 24, 'isVerified': False, 'created': '2017-03-19T20:59:

In [42]:
print (len(all_obelisk_info))
print (len(obelisks))

226
3


# Start processing Data

In [9]:
print(get_ruin_info(158))

{'ruinId': 158, 'bodyName': '1 B', 'ruinTypeName': 'Alpha', 'coordinates': [-28.52, -156.7], 'obelisks': {'b': {'1': 0, '2': 0, '3': 0, '4': 0, '5': 0, '6': 0, '7': 0, '8': 0, '9': 1, '10': 0, '11': 0, '12': 1, '13': 0, '14': 0, '15': 0, '16': 0, '17': 0, '18': 0, '19': 0, '20': 0, '21': 0}, 'c': {'1': 0, '2': 0, '3': 1, '4': 0, '5': 0, '6': 0, '7': 0, '8': 0, '9': 0, '10': 0, '11': 0, '12': 0, '13': 1, '14': 0}, 'd': {'1': 0, '2': 0, '3': 1, '4': 0, '5': 0, '6': 0, '7': 0, '8': 0, '9': 0, '10': 0, '11': 0, '12': 0, '13': 1, '14': 0}, 'e': {'1': 0, '2': 0, '3': 0, '4': 0, '5': 0, '6': 0, '7': 1, '8': 0, '9': 0, '10': 0}, 'h': {'1': 0, '2': 0, '3': 0, '4': 0, '5': 1, '6': 0, '7': 0}, 'i': {'1': 0, '2': 0, '3': 0, '4': 0, '5': 0, '6': 0, '7': 1, '8': 0, '9': 0, '10': 0, '11': 0, '12': 0, '13': 0, '14': 0, '15': 0, '16': 0, '17': 0, '18': 0, '19': 0, '20': 0}, 'j': {'1': 0, '2': 0, '3': 0, '4': 0, '5': 0, '6': 0, '7': 1, '8': 0}, 'k': {'1': 0, '2': 0, '3': 0, '4': 0, '5': 0, '6': 1, '7': 

In [10]:
print(ruins[5]['systemName'])
ruin1 = get_ruin_info(158)
ruin2 = get_ruin_info(156)
print(ruin1['systemCoordinates'])
print(ruin2['systemCoordinates'])

print(ruin_distance(ruin1, ruin2))


COL 173 SECTOR AP-Q B21-2
[8614.19, -116.688, 2733.03]
[8629.19, -98.8125, 2713.06]
30.713749368157746


In [11]:
obelisks_260 = get_active_obelisks(226)


In [85]:
print(obelisks_260)
print(len(obelisks_260))
print([x['codexdataId'] for x in obelisks_260 if x['isBroken'] == False])

[{'codexdataId': 19, 'created': '2017-03-19T20:59:30', 'isVerified': False, 'obeliskgroupId': 40, 'isBroken': False, 'updated': '2017-03-19T20:59:30', 'number': 3, 'id': 640}, {'codexdataId': 18, 'created': '2017-03-19T20:59:30', 'isVerified': True, 'obeliskgroupId': 40, 'isBroken': False, 'updated': '2017-07-11T18:21:04', 'number': 4, 'id': 641}, {'codexdataId': 78, 'created': '2017-03-19T20:59:30', 'isVerified': False, 'obeliskgroupId': 42, 'isBroken': False, 'updated': '2017-07-11T18:21:04', 'number': 3, 'id': 682}, {'codexdataId': 59, 'created': '2017-03-19T20:59:30', 'isVerified': True, 'obeliskgroupId': 43, 'isBroken': False, 'updated': '2017-07-11T18:21:04', 'number': 5, 'id': 700}, {'codexdataId': 100, 'created': '2017-03-19T20:59:30', 'isVerified': True, 'obeliskgroupId': 45, 'isBroken': False, 'updated': '2017-07-11T18:21:04', 'number': 13, 'id': 733}, {'codexdataId': 17, 'created': '2017-03-19T20:59:30', 'isVerified': True, 'obeliskgroupId': 47, 'isBroken': False, 'updated':

In [13]:
print(get_obelisk_group_codex_data(15))

[19, 18, 78, 59, 100, 17, 15, 59, 99, 101, 33, 37, 95, 97, 36, 56]


In [14]:
print(codex_data)
all_codex_data = set([x['id'] for x in codex_data])
print(all_codex_data)

[{'id': 1, 'categoryId': 1, 'artifactId': 3, 'entryNumber': 1, 'text': "Thank you commander, this data has proven vital to my ongoing research. While it seemed logical that the alpha-pattern data corresponded with the core information on the Guardians' biology and ecology, I couldn't be certain until I received the data from your scan. I'm pleased my hypothesis has proved correct, and I've started to unlock some fascinating details about the Guardians. The extracted data relates to individuals rather than their species as a whole, but thanks to the information you have transmitted, I believe I will be able to merge the data to create a general picture.", 'created': '2017-03-19T20:59:29', 'updated': '2017-04-23T16:37:04'}, {'id': 2, 'categoryId': 1, 'artifactId': 6, 'entryNumber': 2, 'text': "The information you've uncovered is very interesting and regards the Guardians' anatomy and physiology, you might be surprised to learn they weren't that different to us. Their morphology was biped

In [15]:
ob15 = get_obelisk_group_codex_data(15)
print (all_codex_data - set(ob15))

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 34, 35, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 57, 58, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 96, 98}
